#    <center>Informatique tc3 (Projet Web) - TD4</center>

## <center style="color: #66d">Développement d'interfaces Web</center>

### 1. Modalité pratiques pour le développement d'interfaces Web

#### a) Via le code source du notebook

Il est possible d'inclure du code HTML directement dans le code source d'une cellule texte d'un notebook.

Essayez : c'est simple, il suffit d'écrire des balises dans une zone de texte <i>(Markdown)</i>.<br>
Si vous manquez <span style="color:red">d'inspiration</span>, vérifiez simplement comment <em>CE TEXTE</em> a été mis en italiques, ou <strong>CELUI-CI</strong> en gras, en regardant simplement le code source de la cellule.

Cette possibilité est pratique pour modifier facilement la présentation du texte d'un notebook, mais ne permettra pas de développer facilement une interface pour une application.

#### b) Via le module IPython HTML

Cette seconde méthode est nettement plus adaptée pour tester une interface en cours de développement. C'est la méthode préconisée tout au long de ce TD

In [1]:
from IPython.core.display import HTML

source_code = ' \
  <div style="background-color: #fef; border: 1px solid #cac; border-radius: 5px; padding: 0.5em"> \
  texte généré via python \
  </div> \
'
HTML(source_code)

#### c) En développant vos propres pages html

<div style="background-color:#fee;padding:10px;border-radius:3px;margin-top:1.33em">
C'est la méthode que vous devrez obligatoirement utiliser pour votre projet.
</div>

### 2. Interface de choix d'une image

La base de données <tt>rtserver.sqlite</tt> fournie pour ce TD contient les trois scènes déjà vues lors du TD précédent, et les images correspondantes se trouvent dans le répertoire <tt>client/images</tt>.

<div style="background-color:#fee;padding:10px;border-radius:3px;margin-top:1.33em">
__N.B.__ Pour obtenir le fonctionnement requis, il est absolument nécessaire d'utiliser le serveur <tt>rtserver.py</tt> fourni pour ce TD, qui implémente l'ensemble des développements suggérés pour le TD précédent, voire un peu plus...
</div>

#### 2.1 Affichage d'une image obtenue via le serveur de TD

__Q1. Afficher une image :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Démarrer le serveur et afficher dans le notebook une des trois images fournies pour ce TD, via le module HTML et une balise <tt>&lt;img&gt;</tt> appropriée, faisant appel au serveur pour obtenir l'image...
<br><br>
<b>N.B.</b> Pour que l'image soit servie par votre serveur (et non le serveur de notebook) il faut que son URL commence par <tt><span>http:</span>//localhost:8080/</tt>.<br>Ne pas oublier non plus que le répertoire <tt>client</tt> n'apparait pas dans l'URL des documents statiques.
</div>

#### 2.2 Affichage d'une liste de choix de scène

Le serveur a été muni d'un service permettant d'obtenir la liste des scènes formattée en HTML de manière à être employée facilement dans une liste de sélection.

__Q2. Obtenir la liste des scènes :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Récupérer la fonction <i>wget()</i> utilisée lors du TD précédent et la modifier le cas échéant pour visualiser
la réponse du serveur à une requête <tt>GET</tt> à l'adresse <tt>/service/image</tt> avec une entête HTTP <tt>Accept:&nbsp;text/html</tt>
<br>
<br>
__N.B.__ pour afficher la réponse, il sera nécessaire de la convertir au format texte par un appel à la fonction <tt>str(reponse,'utf-8')</tt>
</div>

In [15]:
import http.client
import json

def request(method,url,body='',headers={}):
    conn = http.client.HTTPConnection('localhost:8080')
    conn.request(method,url,body,headers)
    response = conn.getresponse()
    if not response.status == 200:
        print("{} - {}".format(response.status,response.reason))
    else:
        content = response.read().decode("utf-8")
        try:
            content = json.loads(response.read().decode("utf-8"))
        except json.decoder.JSONDecodeError:
            pass
        finally:
            return content

In [17]:
data = json.loads(request("GET", "/service/scene/test_cube"))
filename = data["filename"]

In [20]:
HTML("<img src='%s' style='margin:auto'/>" % filename)

__Q3. Afficher une liste de sélection avec les scènes :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Toujours à l'aide du module HTML, afficher une liste de sélection (balise <tt>select</tt>) dont le contenu sera obtenu par
une requête à l'adresse précédente.
</div>

In [28]:
html = "<select onchange='image.src=this.value'>"
scenes = json.loads(request("GET", "/service/scene"))
for scene in scenes:
    html += "<option value='%s'>%s</option>" % (scene["filename"], scene["name"])
html += "</select>"
html += "<br>"
html += "<img id='image'>"
HTML(html)

#### 2.3 Affichage dynamique d'une image

Pour afficher automagiquement l'image sélectionnée, un peu de code javascript est nécessaire.

__Q4. Afficher dynamiquement l'image choisie :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Réitérer le travail effectué pour la question 2, puis :
<ul style="margin-top:0">
<li style="margin-top:0">ajouter au code HTML une balise <tt>&lt;img id="image1"&gt;</tt>,
<li style="margin-top:0">ajouter à la balise <tt>select</tt> un attribut <tt>onchange="image1.src = '<span>http:</span>//localhost:8080'+this.value"</tt>
</ul>
<br>
et profiter du résultat...
</div>

## 3. Modification et recalcul de la scène

Cette partie était facile, car le code renvoyé par le serveur était du code HTML. Il suffisait donc de le récupérer et de l'intégrer tel quel dans l'interface.

Pour modifier une scène, il va falloir récupérer sa description sérialisée au format JSON (cf. TD précédent) et la désérialiser en javascript pour obtenir un objet (en javascript les dictionnaires et les objets sont équivalents).
La modification de cet objet pourra ensuite être effectuée (toujours en javascript). Pour l'enregistrer en base de données et générer l'image correspondante il faudra ensuite re-sérialiser l'objet et soumettre la modification au serveur.

On se propose dans la suite de créer une interface opérationnelle permettant de modifier la couleur de la source de lumière d'une scène donnée.

### 3.1 Mise en place de l'interface

Le code HTML est classique. Il comporte une image, trois actionneurs permettant de modifier les trois composantes de couleur, et des zones de texte pour afficher le nom de la couleur et plus tard sa valeur :

In [5]:
img_style = 'style="margin-bottom: 1em; width: 200px; height: 150px; border: 1px solid #ccc;"'
input_style = 'style="width:200px; display: inline; position: relative; top: 5px"'
span_style = 'style="display: inline-block; width: 4em;"'
html = '\
<img id="image2" %ims> \
<input type="range" id="rin" min="0" max="1" step="0.1" %is> <span %ss>rouge</span><span id="rin_value"></span><br> \
<input type="range" id="gin" min="0" max="1" step="0.1" %is> <span %ss>vert</span><span id="gin_value"></span><br> \
<input type="range" id="bin" min="0" max="1" step="0.1" %is> <span %ss>bleu</span><span id="bin_value"></span><br> \
'.replace('%ims',img_style).replace('%is',input_style).replace('%ss',span_style)
HTML(html)

### 3.2 Chargement de l'image

Le chargement de l'image se fait comme précédemment, au détail près que l'on a maintenant nommé la fonction <tt>update_image</tt> permettant d'afficher l'image à l'obtention de la réponse HTTP.

Noter également que l'on construit l'URL de l'image à charger en ajoutant une chaîne de requête aléatoire (cf. la ligne javascript avec <tt>Math.random()</tt>), afin d'obliger le navigateur à effectivement recharger l'image à chaque fois que l'on exécute ce code. Sans cette astuce, comme le nom de l'image ne change pas, le navigateur ne la rechargerait pas après qu'elle ait été recalculée.

In [29]:
html = ' \
<script> \
  var localhost = "http://localhost:8080"; \
  var r = new XMLHttpRequest(); \
  update_image = function() { \
    data = JSON.parse(this.responseText); \
    scene = JSON.parse(data.serial);\
    image2.src = localhost + data.url + "?" + Math.random(); \
    rin_value.innerHTML = rin.value = scene.sources[0].color.r.toFixed(1); \
    gin_value.innerHTML = gin.value = scene.sources[0].color.g.toFixed(1); \
    bin_value.innerHTML = bin.value = scene.sources[0].color.b.toFixed(1); \
  }; \
  r.onload = update_image; \
  r.open("GET",localhost + "/service/scene/1",true); \
  r.send(); \
</script>'
HTML(html)

### 3.3 Interactivité et recalcul de l'image

L'interactivité se met en place en écrivant un gestionnaire d'événement pour les trois actionneurs. Lors d'un déplacement de l'un des curseurs, le gestionnaire d'événement doit :
<ul style="margin-top:0">
<li style="margin-top:0">mettre à jour la valeur affichée dans le <tt>span</tt>,
<li style="margin-top:0">et soumettre une requête au serveur pour recalculer l'image.
</ul>

Lorsque l'image aura été recalculée, c'est à dire à l'obtention de la réponse HTTP, il suffira d'appeler la fonction <tt>update_image</tt> :

In [30]:
html = ' \
<script> \
  rin.onchange = bin.onchange = gin.onchange = function() { \
    scene.sources[0].color.r = parseFloat(rin.value); \
    scene.sources[0].color.g = parseFloat(gin.value); \
    scene.sources[0].color.b = parseFloat(bin.value); \
    body = JSON.stringify({operation: "update", serial:JSON.stringify(scene)}); \
    var r = new XMLHttpRequest(); \
    r.onload = update_image; \
    r.open("POST",localhost + "/service/scene/1",true); \
    r.send(body); \
    console.log(body); \
  }; \
</script> \
'
HTML(html)

Vérifiez en déplaçant les curseurs, maintenant l'image se met à jour !

## 4. Passage en vraie grandeur

Si l'écriture d'une interface Web via un notebook Python est possible (nous venons de le démontrer) il n'en reste pas moins que cette approche ne permet pas de développer une application à part entière.

Pour cela il va falloir créer un document html avec le code html, css, et javascript mis en oeuvre au long du chapitre 3. ci-dessus. Votre document s'appelera <tt>eclairage.html</tt> et se trouvera dans le répertoire <tt>/client</tt>.

__Q5. Développer un document html pour modifier l'éclairage de la scène n°1 :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Récupérer et mettre en forme le code présenté ci-dessus pour en faire un document autonome exécutable directement dans un navigateur.
<br><br>
Si tout se passe bien, il devrait fonctionner à l'adresse <a href="http://localhost:8080/eclairage.html">http://localhost:8080/eclairage.html</a>.
</div>

__Q6. Développer un document html pour modifier l'éclairage des scènes connues :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Après avoir effectué une copie du précédent cette fois-ci nommée <tt>eclairage_scenes.html</tt>,
modifier le programme pour récupérer la liste des scènes et pouvoir choisir celle dont on modifie l'éclairage.
<br><br>
Si tout se passe bien, il devrait fonctionner à l'adresse <a href="http://localhost:8080/eclairage_scenes.html">http://localhost:8080/eclairage_scenes.html</a>.
</div>


<img src="eclairage.png" width="300">
<center><a href="eclairage.png">Exemple d'interface</a></center>

In [8]:
# Initialisation de la base de données
import sqlite3
conn = sqlite3.connect('rtserver.sqlite')
c = conn.cursor()

c.execute("DROP TABLE IF EXISTS scene")
c.execute("CREATE TABLE scene ( \
  id INTEGER PRIMARY KEY, \
  name TEXT UNIQUE NOT NULL, \
  serial TEXT NOT NULL, \
  width INTEGER,  \
  height INTEGER, \
  ptime REAL, \
  filename TEXT )")

plane = [
    "test_checkered_plane",
    '{"_class_": "Scene", "name": "test_checkered_plane", "ambient": {"_class_": "rgb", "r": 0, "g": 0, "b": 0}, \
"max_bounce": 3, "faraway": 1e+39, "sources": [{"_class_": "LightSource", "position": {"_class_": "vec3", "x": 0, \
"y": 10, "z": -5}, "color": {"_class_": "rgb", "r": 1, "g": 1, "b": 1}}], "objects": [{"_class_": "CheckeredPlane", \
"position": {"_class_": "vec3", "x": 0, "y": -0.5, "z": 0}, "normal": {"_class_": "vec3", "x": 0.0, "y": 1.0, \
"z": 0.0}, "diffuse": [{"_class_": "rgb", "r": 0.1, "g": 0.1, "b": 0.1}, {"_class_": "rgb", "r": 0.9, "g": 0.9, \
"b": 0.9}], "ambient": [{"_class_": "rgb", "r": 1.0, "g": 1.0, "b": 1.0}, {"_class_": "rgb", "r": 1.0, "g": 1.0, \
"b": 1.0}], "specular": [{"_class_": "rgb", "r": 0.5, "g": 0.5, "b": 0.5}, {"_class_": "rgb", "r": 0.5, "g": 0.5, \
"b": 0.5}], "phong": [70, 70], "mirror": [{"_class_": "rgb", "r": 0.5, "g": 0.5, "b": 0.5}, {"_class_": "rgb", \
"r": 0.5, "g": 0.5, "b": 0.5}], "selector": \
"def selector(self,M):\\n  return ((M.x * 2 - 1000.5).astype(int) % 2) == ((M.z * 0.5 + 1000.5).astype(int) % 2)"}], \
"width": 400, "height": 300, "camera": {"_class_": "vec3", "x": 0.0, "y": 0.1, "z": -10.0}}',
    400, 300,
    0.062,
    "client/images/test_checkered_plane.png" 
]
cube = [
    "test_cube",
    '{"_class_": "Scene", "name": "test_cube", "ambient": {"_class_": "rgb", "r": 0, "g": 0, "b": 0}, \
"max_bounce": 3, "faraway": 1e+39, "sources": [{"_class_": "LightSource", "position": {"_class_": "vec3", \
"x": 0, "y": 0, "z": -5}, "color": {"_class_": "rgb", "r": 1, "g": 1, "b": 1}}], "objects": [{"_class_": \
"Cube", "center": {"_class_": "vec3", "x": 0, "y": -0.1, "z": 1}, "U": {"_class_": "vec3", "x": 0.5000000000000001,\
"y": 0.7071067811865476, "z": -0.5000000000000001}, "V": {"_class_": "vec3", "x": 0.7071067811865476, "y": 0.0, \
"z": 0.7071067811865476}, "width": 0.4, "diffuse": [{"_class_": "rgb", "r": 0.5, "g": 0, "b": 0.5}, {"_class_": "rgb",\
"r": 0.5, "g": 0.5, "b": 0}, {"_class_": "rgb", "r": 0, "g": 0.5, "b": 0.5}, {"_class_": "rgb", "r": 0.75, \
"g": 0, "b": 0}, {"_class_": "rgb", "r": 0, "g": 0.75, "b": 0}, {"_class_": "rgb", "r": 0, "g": 0, "b": 0.75}], \
"ambient": [{"_class_": "rgb", "r": 1.0, "g": 1.0, "b": 1.0}, {"_class_": "rgb", "r": 1.0, "g": 1.0, "b": 1.0}, \
{"_class_": "rgb", "r": 1.0, "g": 1.0, "b": 1.0}, {"_class_": "rgb", "r": 1.0, "g": 1.0, "b": 1.0}, {"_class_": \
"rgb", "r": 1.0, "g": 1.0, "b": 1.0}, {"_class_": "rgb", "r": 1.0, "g": 1.0, "b": 1.0}], "specular": [{"_class_": \
"rgb", "r": 0.5, "g": 0.5, "b": 0.5}, {"_class_": "rgb", "r": 0.5, "g": 0.5, "b": 0.5}, {"_class_": "rgb", "r": 0.5, \
"g": 0.5, "b": 0.5}, {"_class_": "rgb", "r": 0.5, "g": 0.5, "b": 0.5}, {"_class_": "rgb", "r": 0.5, "g": 0.5, \
"b": 0.5}, {"_class_": "rgb", "r": 0.5, "g": 0.5, "b": 0.5}], "phong": [140, 140, 140, 140, 140, 140], "mirror": \
[{"_class_": "rgb", "r": 0.3, "g": 0.3, "b": 0.3}, {"_class_": "rgb", "r": 0.3, "g": 0.3, "b": 0.3}, {"_class_": \
"rgb", "r": 0.3, "g": 0.3, "b": 0.3}, {"_class_": "rgb", "r": 0.3, "g": 0.3, "b": 0.3}, {"_class_": "rgb", \
"r": 0.3, "g": 0.3, "b": 0.3}, {"_class_": "rgb", "r": 0.3, "g": 0.3, "b": 0.3}]}], "width": 400, "height": 300, \
"camera": {"_class_": "vec3", "x": 0.0, "y": 0.1, "z": -10.0}}',
    400, 300,
    0.141,
    "client/images/test_cube.png"
]
polygon = [
    'test_polygon',
    '{"name": "test_polygon", "_class_": "Scene", "sources": [{"_class_": "LightSource", "position": {"_class_": "vec3",\
"x": 0, "y": 10, "z": -5}, "color": {"_class_": "rgb", "r": 1, "g": 1, "b": 1}}], "objects": [{"_class_": "Polygon", "vertices": [[0, 0], [0, 10], [1, 10], [1, 0]],\
"position": {"_class_": "vec3", "x": -0.5, "y": -0.5, "z": 1}, "U": {"_class_": "vec3", "x": 1, "y": 0, "z": 0}, "V":\
{"_class_": "vec3", "x": 0, "y": 0, "z": 1}, "ns": -1, "diffuse": 0.5, "ambient": 1.0, "specular": 0.5, "phong": 70,\
"mirror": 0.5}], "camera": {"_class_": "vec3", "x": 0.0, "y": 0.1, "z": -10.1}}',
    400, 300, 
    0.034,
    'client/images/test_polygon.png'
]

c.execute("INSERT INTO scene VALUES (NULL, ?, ?, ?, ?, ?, ?)", plane)
c.execute("INSERT INTO scene VALUES (NULL, ?, ?, ?, ?, ?, ?)", cube)
c.execute("INSERT INTO scene VALUES (NULL, ?, ?, ?, ?, ?, ?)", polygon)
conn.commit()